# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


In [2]:
#Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv("output_file/cities.csv")
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Outlook,99,CA,1580529178,78,51.50,-107.05,24.28,4.29
1,Saint George,1,US,1580528790,38,37.10,-113.58,46.40,1.41
2,Celestun,0,MX,1580529179,91,20.87,-90.40,74.53,20.04
3,Nuuk,100,GL,1580529106,78,64.18,-51.72,12.20,10.29
4,Hilo,90,US,1580529180,69,19.73,-155.09,75.20,4.70


# Humidity Heatmap

In [4]:
#Use the Latitude and Longitude as locations and Humidity as the weight
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()
locations = cities_df[["Latitude", "Longitude"]]


In [5]:
#Create a heat map that displays the humidity for every city from the part I of the homework.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [6]:
# Narrow down the DataFrame to find the ideal weather condition.
ideal_weather_df = cities_df.loc[(cities_df["Max Temperature"] > 60) & (cities_df["Max Temperature"] < 90) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 10), :]

# Drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df['index']
ideal_weather_df.head()  

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,Coquimbo,0,CL,1580529190,82,-29.95,-71.34,62.60,9.17
1,Busselton,0,AU,1580528916,23,-33.65,115.33,78.01,8.59
2,Bengkulu,0,ID,1580529233,56,-3.80,102.27,87.87,1.07
3,Salalah,0,OM,1580529108,52,17.02,54.09,66.20,2.24
4,Araouane,0,ML,1580529286,15,18.90,-3.53,65.75,9.91


# Hotel Map

In [7]:
#Store into variable named hotel_df
hotel_df = pd.DataFrame(ideal_weather_df, columns =['City', 'Country', 'Latitude', 'Longitude'])
hotel_df['Hotel Name']=''
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Coquimbo,CL,-29.95,-71.34,
1,Busselton,AU,-33.65,115.33,
2,Bengkulu,ID,-3.80,102.27,
3,Salalah,OM,17.02,54.09,
4,Araouane,ML,18.90,-3.53,


In [8]:
#creates a list to capture hotel_names 
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try: 
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [9]:
hotel_name

['Enjoy Coquimbo Hotel de la Bahía',
 'Esplanade Hotel',
 'Hotel Santika Bengkulu',
 'Salalah Gardens Hotel',
 nan,
 'Gweta Lodge',
 nan,
 nan,
 'HOTEL CAMINO REAL',
 'Hotel Internacional',
 nan,
 'HOTEL MODELO',
 'Pousada Rosa',
 'Hotel Cristy',
 'Diamond Hotel',
 'Hotel y restaurante "El Diamante"',
 nan,
 'La Rose Blanche Hotel',
 'Hotel Suraj INN',
 'Hotel Los Caracoles',
 'Cabaña La Celia',
 'Hotel Polo Orchid',
 'Sann Htay Motel',
 'Hotel IK Residency',
 nan,
 nan,
 'LERABA HOTEL',
 'Old Mill Hotel',
 'Tropiclub Playa El Cuco',
 'Paramirim Palace Hotel',
 nan,
 'InterTower Hotel',
 'Hotel Royal Tocopilla',
 'Hotel Salto Grande',
 nan]

In [10]:
#adds the list of hotels onto locations and drops any values with nan stored.
hotel_df['Hotel Name']= hotel_name
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Coquimbo,CL,-29.95,-71.34,Enjoy Coquimbo Hotel de la Bahía
1,Busselton,AU,-33.65,115.33,Esplanade Hotel
2,Bengkulu,ID,-3.80,102.27,Hotel Santika Bengkulu
3,Salalah,OM,17.02,54.09,Salalah Gardens Hotel
5,Gweta,BW,-20.18,25.23,Gweta Lodge


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame 
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(markers)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))